<a id='HOME'></a>
# CHAPTER 11 Concurrency and Networks
## 開發與網路

* [11.1 開發](#Concurrency)
* [11.2 網路](#Networks)

Time is nature’s way of keeping everything from happening at once.  
Space is what preventseverything from happening to me.  
— Quotes about Time

先前的範例都是在一台電腦一次運行一次，但是可以利用技術做到分散式計算等更強大的功能。

好處有
* 提升性能，可以不用因為某部分運行較慢而塞住後面的程序
* 提升安全性，同時運作多個相同任務防止意外
* 把程式拆解得更細小更好維護與理解
* 網路傳送與接收數據

---
<a id='Concurrency'></a>
## 11.1 開發
[回目錄](#HOME)

一般而言在計算時速度變慢時多半是因為__I/O___的限制以及__CPU限制__  
I/O的速度比起CPU計算上慢上許多，  
CPU算得較慢時可能是遇到科學相關需要大量計算或是圖形相關的計算導致變慢

程式在開發時一般會有兩種技術，__同步(synchronous)__與__非同步(asynchronous)__

* 同步是指說程式在執行時是一個接著一個，A指令做完才做B指令，最後在C指令
* 非同步是可以讓A B C三種指令同時在一個程式裡執行

__佇列(Queue)__中文也翻作隊列，顧名思義是一種像排隊一樣的概念，  
以生活中的情況為人們一個接一個的從隊伍後面加入排隊，而窗口的服務人員則從最前面的民眾一個接一個處理事務。(http://emn178.pixnet.net/blog/post/93475832-%E4%BD%87%E5%88%97(queue))

本書作者以洗碗流程作為範例，分成兩段工作過程，清洗與烘乾。  
假設一個人做事，可以有兩種做法，
1. 洗完一個盤子就拿去烘乾
2. 先洗完全部的盤子，在統一把盤子烘乾

若要改善效率最快的方法就是找幫手，一人負責洗碗一人負責烘乾，但是這樣會遇到一個問題，如果洗盤子的速度大於烘乾盤子時，是要等待烘乾的人員閒置後在遞交盤子給他，還是先行放置在桌上，在繼續洗下一個盤子，等他有空時再自行拿取，前者就是同步的概念，後者則為非同步的概念。

假設水槽中的盤子們是佇列(Queue)中的工作項目，可以進行同步與分同步的工作流程。  
* 同步：把水槽中的髒盤子給第一個閒置的洗碗人員洗，洗完後等烘乾人員閒置後再把盤子給他
* 非同步：洗盤子的人員洗好就將盤子放置在桌上後繼續清洗下一個，烘乾人員閒置時就去看桌上有無盤子可以清洗。

![Alt text](http://i.imgur.com/mtFun5p.png "時間花費表")

#### 原始洗碗流程

```python
import os
import time
from datetime import datetime

def washer(dishes, now_):
	for dish in dishes:
		now = datetime.now()
		print('Washing', dish, ', time:', now - now_, ', pid', os.getpid())
		time.sleep(1)
		dryer(dish, now_)

def dryer(dish, now_):
	now = datetime.now()
	print('Drying ', dish, ', time:', now - now_, ', pid', os.getpid())
	time.sleep(2)

if __name__ == "__main__":
	now_ = datetime.now()
	dishes = ['dish-1', 'dish-2', 'dish-3', 'dish-4']
	washer(dishes, now_)
```

In [1]:
import subprocess
ret = subprocess.getoutput('python Data/dishes.py')
print(ret) #執行需時間，請等候

Washing dish-1 , time: 0:00:00 , pid 13204
Drying  dish-1 , time: 0:00:01.010387 , pid 13204
Washing dish-2 , time: 0:00:03.023734 , pid 13204
Drying  dish-2 , time: 0:00:04.027234 , pid 13204
Washing dish-3 , time: 0:00:06.027493 , pid 13204
Drying  dish-3 , time: 0:00:07.039876 , pid 13204
Washing dish-4 , time: 0:00:09.052473 , pid 13204
Drying  dish-4 , time: 0:00:10.060635 , pid 13204


### 行程or處理程序(Processes)

[wiki 行程](https://zh.wikipedia.org/wiki/%E8%A1%8C%E7%A8%8B)

下列範例可以模擬兩個人員在進行分工合作，洗碗為主行程，烘乾則為另開的行程，所以洗碗者不必等到烘乾完畢就可以洗下一個盤子

```python
import multiprocessing as mp
import os
import time
from datetime import datetime

def washer(dishes, output, now_):
	for dish in dishes:
		now = datetime.now()
		print('Washing', dish, ', time:', now - now_, ', pid', os.getpid())
		time.sleep(1)
		#把東西丟給其他行程後繼續執行下一個
		output.put(dish)

def dryer(input, now_):
	while True:
		dish = input.get()
		now = datetime.now()
		print('Drying ', dish, ', time:', now - now_, ', pid', os.getpid())
		time.sleep(2)
		input.task_done()

if __name__ == "__main__":
	now_ = datetime.now()
	#建立佇列
	dish_queue = mp.JoinableQueue()
	#創建行程(烘乾人員)
	dryer_proc = mp.Process(target=dryer, args=(dish_queue, now_,))
	dryer_proc.daemon = True
	#啟動行程(上班囉)
	dryer_proc.start()
	#time.sleep(1)
	
	dishes = ['dish-1', 'dish-2', 'dish-3', 'dish-4']
	washer(dishes, dish_queue, now_)
	dish_queue.join()
```
p.s. 在ipython中非主程式的行程print不出來，請自行在本機端cmd跑，或是把print改成寫成實體檔案方可看見結果，  
Data資料夾中有dishes_process.py檔案可供使用  

結果:  
Washing dish-1 , time: 0:00:00.037144 , pid 10480  
Washing dish-2 , time: 0:00:01.047415 , pid 10480  
Drying  dish-1 , time: 0:00:01.047415 , pid 7280  
Washing dish-3 , time: 0:00:02.060229 , pid 10480  
Drying  dish-2 , time: 0:00:03.047613 , pid 7280  
Washing dish-4 , time: 0:00:03.063241 , pid 10480  
Drying  dish-3 , time: 0:00:05.047959 , pid 7280  
Drying  dish-4 , time: 0:00:07.053659 , pid 7280

### 執行緒(Threads)

[wiki 執行緒](https://zh.wikipedia.org/wiki/%E7%BA%BF%E7%A8%8B)

下列範例可以模擬一個人員洗，兩個人烘的分工合作，全部都為同一個主行程，但是另開兩個線程來處理烘乾工作

```python

import threading, queue
import os
import time
from datetime import datetime

def washer(dishes, dish_queue, now_):
	for dish in dishes:
		now = datetime.now()
		print ("Washing", dish, now - now_, ', pid', os.getpid(), threading.current_thread())
		time.sleep(1)
		dish_queue.put(dish)

def dryer(dish_queue, now_):
	while True:
		dish = dish_queue.get()
		now = datetime.now()
		print ("Drying ", dish, now - now_, ', pid', os.getpid(), threading.current_thread())
		time.sleep(2)
		dish_queue.task_done()

if __name__ == "__main__":
	dish_queue = queue.Queue()
	now_ = datetime.now()
    #控制要開幾條執行緒
	for n in range(2):
		dryer_thread = threading.Thread(target=dryer, args=(dish_queue, now_))
		dryer_thread.daemon = True
		dryer_thread.start()
		
	dishes = ['dishe-1', 'dishe-2', 'dishe-3', 'dishe-4']
	washer(dishes, dish_queue, now_)
	dish_queue.join()

```

In [7]:
import subprocess
ret = subprocess.getoutput('python Data/dishes_threads.py')
print(ret) #執行需時間，請等候

Washing dishe-1 0:00:00.000505 , pid 5376 <_MainThread(MainThread, started 18348)>
Washing dishe-2 0:00:01.012149 , pid 5376 <_MainThread(MainThread, started 18348)>
Drying  dishe-1 0:00:01.012149 , pid 5376 <Thread(Thread-1, started daemon 11096)>
Washing dishe-3 0:00:02.012222 , pid 5376 <_MainThread(MainThread, started 18348)>
Drying  dishe-2 0:00:02.012222 , pid 5376 <Thread(Thread-2, started daemon 8768)>
Washing dishe-4 0:00:03.022536 , pid 5376 <_MainThread(MainThread, started 18348)>
Drying  dishe-3 0:00:03.022536 , pid 5376 <Thread(Thread-1, started daemon 11096)>
Drying  dishe-4 0:00:04.032120 , pid 5376 <Thread(Thread-2, started daemon 8768)>


總而言之，對於 Python，建議如下：
* 使用執行緒來解決 I/O 限制問題；
* 使用行程、網絡或者事件（下一節會介紹）來處理 CPU 限制問題。

書中還介紹許多函示庫可以辦到各種不同的佇列(Queue)功能
* gevent
* twisted
* asyncio
* Redis

---
<a id='Networks'></a>
## 11.2 網路
[回目錄](#HOME)